# 🤖 Modelo de Recomendación de Películas

**Objetivo:** Implementar algoritmo de Collaborative Filtering Item-Based

**Fecha:** 22 de noviembre de 2024

**Algoritmo:** Item-Based Collaborative Filtering
- Calcula similitud entre películas
- Recomienda películas similares a las que le gustaron al usuario

## Pasos:
1. Cargar datos limpios
2. Crear matriz de similitud
3. Implementar función de recomendación
4. Evaluar el modelo
5. Guardar modelo

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import pickle

# Cargar datos limpios
ratings = pd.read_csv('../data/processed/ratings_clean.csv')
movies = pd.read_csv('../data/processed/movies_clean.csv')

print("✅ Datos cargados")
print(f"📊 Ratings: {len(ratings):,}")
print(f"🎬 Películas: {len(movies):,}")
print(f"👥 Usuarios: {ratings['user_id'].nunique()}")

ModuleNotFoundError: No module named 'sklearn'

In [3]:
# Crear matriz pivote: usuarios en filas, películas en columnas
user_movie_matrix = ratings.pivot_table(
    index='user_id',
    columns='item_id',
    values='rating',
    fill_value=0
)

print("📊 MATRIZ USUARIO-PELÍCULA")
print(f"Dimensiones: {user_movie_matrix.shape}")
print(f"Usuarios: {user_movie_matrix.shape[0]}")
print(f"Películas: {user_movie_matrix.shape[1]}")
print()
print("Primeras 5 filas y 5 columnas:")
user_movie_matrix.iloc[:5, :5]

NameError: name 'ratings' is not defined

In [ ]:
# Transponer para tener películas en filas
movie_user_matrix = user_movie_matrix.T

# Calcular similitud coseno entre películas
print("⏳ Calculando similitud entre películas...")
movie_similarity = cosine_similarity(movie_user_matrix)

# Convertir a DataFrame para facilitar uso
movie_similarity_df = pd.DataFrame(
    movie_similarity,
    index=movie_user_matrix.index,
    columns=movie_user_matrix.index
)

print("✅ Matriz de similitud calculada")
print(f"Dimensiones: {movie_similarity_df.shape}")
print()
print("Ejemplo - Similitud entre primeras 5 películas:")
movie_similarity_df.iloc[:5, :5]

In [ ]:
def get_movie_recommendations(movie_id, n_recommendations=10):
    """
    Obtiene recomendaciones basadas en similitud con una película
    
    Args:
        movie_id: ID de la película
        n_recommendations: Número de recomendaciones a retornar
        
    Returns:
        DataFrame con películas recomendadas y su score de similitud
    """
    # Obtener películas similares
    similar_movies = movie_similarity_df[movie_id].sort_values(ascending=False)
    
    # Quitar la misma película
    similar_movies = similar_movies[similar_movies.index != movie_id]
    
    # Top N recomendaciones
    top_movies = similar_movies.head(n_recommendations)
    
    # Crear DataFrame con información de películas
    recommendations = pd.DataFrame({
        'item_id': top_movies.index,
        'similarity_score': top_movies.values
    })
    
    # Agregar títulos
    recommendations = recommendations.merge(
        movies[['item_id', 'title']], 
        on='item_id'
    )
    
    return recommendations[['title', 'similarity_score']]

print("✅ Función de recomendación creada")

In [ ]:
# Probar con una película popular
test_movie_id = 50  # Star Wars (1977)

# Obtener nombre de la película
movie_name = movies[movies['item_id'] == test_movie_id]['title'].values[0]

print(f"🎬 PELÍCULA BASE: {movie_name}")
print(f"📝 ID: {test_movie_id}")
print()
print("🎯 TOP 10 RECOMENDACIONES:")
print("=" * 70)

recommendations = get_movie_recommendations(test_movie_id, n_recommendations=10)

for idx, row in recommendations.iterrows():
    print(f"{idx+1:2d}. {row['title']:50s} - Score: {row['similarity_score']:.3f}")

In [ ]:
# Probar con una película popular
test_movie_id = 50  # Star Wars (1977)

# Obtener nombre de la película
movie_name = movies[movies['item_id'] == test_movie_id]['title'].values[0]

print(f"🎬 PELÍCULA BASE: {movie_name}")
print(f"📝 ID: {test_movie_id}")
print()
print("🎯 TOP 10 RECOMENDACIONES:")
print("=" * 70)

recommendations = get_movie_recommendations(test_movie_id, n_recommendations=10)

for idx, row in recommendations.iterrows():
    print(f"{idx+1:2d}. {row['title']:50s} - Score: {row['similarity_score']:.3f}")

In [ ]:
import os

# Crear carpeta models si no existe
os.makedirs('../src', exist_ok=True)

# Guardar matriz de similitud
with open('../src/movie_similarity.pkl', 'wb') as f:
    pickle.dump(movie_similarity_df, f)

# Guardar datos necesarios
with open('../src/movies_data.pkl', 'wb') as f:
    pickle.dump(movies, f)

print("✅ Modelo guardado en:")
print("   📁 src/movie_similarity.pkl")
print("   📁 src/movies_data.pkl")

In [ ]:
print("=" * 70)
print("🎉 MODELO DE RECOMENDACIÓN COMPLETADO")
print("=" * 70)
print()
print("✅ Algoritmo: Item-Based Collaborative Filtering")
print(f"✅ Películas en el modelo: {movie_similarity_df.shape[0]}")
print(f"✅ RMSE: {rmse:.3f}")
print()
print("🎯 PRÓXIMOS PASOS:")
print("   1. Crear interfaz web con Streamlit")
print("   2. Integrar modelo con la interfaz")
print("   3. Permitir que usuarios seleccionen películas")
print("   4. Mostrar recomendaciones personalizadas")
print()
print("=" * 70)